In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import os
from ipywidgets import FloatProgress
from IPython.display import display
from time import time
import sys
sys.path.append("/home/pg1g15/Documents/SEM_Container")
import SEM
from numba import jit

In [2]:
Header = ["scale", "id", "desc_scale", "desc_id", "num_prog", "pid", "upid", "desc_pid", "phantom", "sam_mvir", "mvir", "rvir", "rs", "vrms", "mmp?", "scale_of_last_MM", "vmax", "x", "y", "z", "vx", "vy", "vz", "Jx", "Jy", "Jz", "Spin", "Breadth_first_ID", "Depth_first_ID", "Tree_root_ID", "Orig_halo_ID", "Snap_num", "Next_coprogenitor_depthfirst_ID", "Last_progenitor_depthfirst_ID", "Last_mainleaf_depthfirst_ID", "Tidal_Force", "Tidal_ID", "Rs_Klypin", "Mvir_all", "M200b", "M200c", "M500c", "M2500c", "Xoff", "Voff", "Spin_Bullock", "b_to_a", "c_to_a", "Ax", "Ay", "Az", "b_to_a500c", "c_to_a500c", "Ax500c", "Ay500c", "Az500c", "T/|U|", "M_pe_Behroozi", "M_pe_Diemer" ]

In [3]:
Files = os.listdir("./CSVed")
print(len(Files))

24674


In [4]:
@jit
def GroupIt():
    Groups = []
    f = FloatProgress(min = 0, max = len(Files))
    display(f)
    for File in Files:
        Groups = []
        #builds and groups the data
        df  = pd.read_csv("./CSVed/%s" %(File), header = None, names = Header)
        grouped = df.groupby('scale')

        #Makes a list of all the groups
        for name, group in grouped:
            Groups.append(name)
        Groups.reverse()

        Outfile = open("./SEMunbinned/%s" %(File), "w")
        Outfile.write("START\n")
        for i in range(len(Groups) - 1): 
            NextStep = grouped.get_group(Groups[i])
            CurrentStep = grouped.get_group(Groups[i + 1])

            NextMproID = NextStep.iloc[0]['id'].astype(int)  

            Outfile.write("mpro " + str(CurrentStep.iloc[0]['scale']) +" "+ str(CurrentStep.iloc[0]['M200c']) +" "+ str(CurrentStep.iloc[0]['rvir']) +" "+ str(CurrentStep.iloc[0]['rs'])+"\n")

            for index, Halo in CurrentStep.iterrows():
                #print(Halo)
                if Halo['pid'] == -1:
                    NextID = Halo['desc_id'].astype(int)
                    #print(NextID)
                    This = NextStep[NextStep['id'] == NextID]
                    if This.iloc[0]['pid'] == NextMproID:


                        Outfile.write( "sat " + str(Halo['scale']) +" "+ str(Halo['M200c']) +" "+ str(Halo['rvir']) +" "+ str(Halo['rs']) + "\n")
        f.value += 1

In [5]:
GroupIt()

The installed widget Javascript is the wrong version. It must satisfy the semver range ~2.1.4.


In [6]:
BadTrees = []
SaveFile = "/home/pg1g15/Documents/SEM_Container/LinearTime100Res11/"
def BinTrees(TreeFiles): 
    StartTime = time()
    TimeStep = 0
    RedshiftStep = 0
    MassAtLastWrite = 0
    ResolvedMass = -300
    Resolution = 10**8
    Switch = True
    Exception = False
    f = FloatProgress(min = 0, max = len(Files))
    display(f)
    for File in TreeFiles:
        """
        print(File)
        if(File == 'Tree91.dat'):
            continue
        if(File == 'Tree103.dat'):
            continue
        if(File == 'Tree47.dat'):
            continue
        """
        Switch = True
        RedshiftStep = 0
        MassAtLastWrite = 0
        ResolvedMass = -300
        f.value += 1
        with open("./SEMunbinned/%s" %(File), "r") as InputFile, open((SaveFile + "%s" %(File)), "w") as OutputFile:
            for LineStr in InputFile:
                Line = str.split(LineStr) 
                if Line[0] == "START":
                    OutputFile.write("START\n")
                else:
                    Line[1] = float(Line[1])
                    Line[1] = (1 / Line[1]) - 1
                    L1asT = SEM.RedshiftToTime(Line[1])
                    Line[2] = float(Line[2])

                    if Line[0] == "mpro":
                        if Switch == True:
                            OutputFile.write( "mpro  %s  %s  %s  %s\n" %(Line[1], math.log10(Line[2]), Line[3], Line[4]) )
                            MassAtLastWrite = Line[2]
                            Switch = False
                        elif L1asT >= (TimeStep + 0.1):
                            if MassAtLastWrite - Line[2] - 10**ResolvedMass > 0:                            
                                OutputFile.write( "1  acc  %s  %s  1\n" %(Line[1], math.log10(MassAtLastWrite - Line[2] - 10**ResolvedMass) ) ) 
                            else:
                                OutputFile.write( "1  acc  %s  %s  -1\n" %(Line[1], math.log10(abs(MassAtLastWrite - Line[2] - 10**ResolvedMass ) ) ) )
                            MassAtLastWrite = Line[2]
                            ResolvedMass = 0
                            try:
                                OutputFile.write( "mpro  %s  %s  %s  %s\n" %(Line[1], math.log10(Line[2]), Line[3], Line[4]) )
                            except:
                                print(File)
                                BadTrees.append(int(File[4:-4]))
                                Exception = True
                                break
                            RedshiftStep += 0.01
                    elif Line[0] == "sat" and Line[2] > Resolution:
                        ResolvedMass = math.log10(10**ResolvedMass + Line[2])
                        OutputFile.write( "1  %s  %s  %s  %s\n" %(Line[1], math.log10(Line[2]), Line[3], Line[4]) )
        if(Exception == True):
            Exception = False
            continue
    print(BadTrees)
    np.savetxt(SaveFile + "BadTrees.txt", np.array(BadTrees).astype(int))

In [7]:
BinTrees(Files)

The installed widget Javascript is the wrong version. It must satisfy the semver range ~2.1.4.


Tree18075.dat
Tree5595.dat
Tree3497.dat
Tree2558.dat
Tree13863.dat
Tree436.dat
Tree7279.dat
Tree361.dat
Tree2164.dat
Tree8499.dat
Tree8203.dat
Tree3694.dat
Tree14035.dat
Tree5364.dat
Tree2445.dat
Tree10968.dat
Tree12027.dat
Tree1167.dat
Tree10589.dat
Tree10278.dat
Tree13025.dat
Tree12029.dat
Tree8784.dat
Tree1277.dat
Tree3058.dat
Tree10343.dat
Tree229.dat
Tree17331.dat
Tree16329.dat
Tree7585.dat
Tree8877.dat
Tree2952.dat
Tree13108.dat
Tree786.dat
Tree16278.dat
Tree6618.dat
Tree4545.dat
Tree1287.dat
Tree7333.dat
Tree10901.dat
Tree2223.dat
Tree994.dat
Tree7378.dat
Tree11555.dat
Tree7004.dat
Tree19488.dat
Tree7028.dat
Tree9507.dat
Tree4975.dat
Tree10268.dat
Tree10635.dat
Tree10611.dat
Tree4630.dat
Tree13728.dat
Tree4714.dat
Tree13685.dat
Tree705.dat
Tree1309.dat
Tree236.dat
Tree21571.dat
Tree9866.dat
Tree21912.dat
Tree1718.dat
Tree5608.dat
Tree6565.dat
Tree7291.dat
Tree773.dat
Tree11752.dat
Tree15719.dat
Tree4889.dat
Tree10879.dat
Tree5054.dat
Tree3558.dat
Tree7419.dat
Tree10421.dat
Tree4

When you are COMPLETELY DONE with the data files and have trees in SEM_Container

In [21]:
%%bash

cd /data/B-P50Mpc/Above12/RedshiftCut/CSVed

find . -name "*.dat" -print0 | xargs -0 rm

cd /data/B-P50Mpc/Above12/RedshiftCut/SEMunbinned

find . -name "*.dat" -print0 | xargs -0 rm

In [20]:
np.loadtxt("/home/pg1g15/Documents/SEM_Container/LinearTime100/BadTrees.txt").astype(int)

array([ 634, 1230,  222,  493,  413])